### python 2.x to run

In [9]:
import numpy as np

In [11]:
def loadDataSet():
    return [[1,3,4] , [2,3,5] , [1,2,3,5] , [2,5]]

def createC1(dataSet):
    C1 = []
    
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    
    return map(frozenset , C1)



In [12]:
def scanD(D , Ck , minSupport): #最小支持度
    ssCnt = {}
    
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can):
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
                
    numItems = float(len(D))
    retList = []
    supportData = {}
    
    for key in ssCnt:
        support = ssCnt[key] / numItems
        
        if support>= minSupport:
            retList.insert(0 , key)
        supportData[key] = support
    
    return retList , supportData

In [13]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [14]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [15]:
L1 , suppData0 = scanD(D , C1 , 0.5)

In [16]:
L1

[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]

In [17]:
suppData0

{frozenset({4}): 0.25,
 frozenset({5}): 0.75,
 frozenset({2}): 0.75,
 frozenset({3}): 0.75,
 frozenset({1}): 0.5}

In [18]:
[L1]

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]]

In [19]:
def aprioriGen(Lk , k):
    retList = []
    lenLk = len(Lk)
    
    for i in range(lenLk):
        for j in range(i+1 , lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    
    return retList

def apriori(dataSet , minSupport = 0.5):
    C1 = createC1(dataSet)
    D = map(set , dataSet)
    L1 , supportData = scanD(D , C1 , minSupport)
    L = [L1]
    
    k = 2
    
    while(len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2] , k)
        Lk , supK = scanD(D , Ck , minSupport)
        supportData.update(supK)
        L.append(Lk)
        
        k += 1
        
    return L , supportData

In [21]:
a=[1,2,3]
a[:0] == a[:0]

True

In [22]:
L , suppData0 = apriori(dataSet)

In [23]:
L

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})],
 [frozenset({2, 3, 5})],
 []]

In [24]:
suppData0

{frozenset({5}): 0.75,
 frozenset({3}): 0.75,
 frozenset({2, 3, 5}): 0.5,
 frozenset({1, 2}): 0.25,
 frozenset({1, 5}): 0.25,
 frozenset({3, 5}): 0.5,
 frozenset({4}): 0.25,
 frozenset({2, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({1}): 0.5,
 frozenset({1, 3}): 0.5,
 frozenset({2}): 0.75}

In [25]:
aprioriGen(L[0] , 2)

[frozenset({1, 3}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({3, 5}),
 frozenset({2, 5})]

In [26]:
def generateRules(L , supportData , minConf = 0.7): #关联规则的可信度
    bigRuleList = []
    
    for i in range(1 , len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            
            if i>1:
                rulesFromConseq(freqSet , H1 , supportData , bigRuleList , minConf)
            else:
                calcConf(freqSet , H1 , supportData , bigRuleList , minConf)
            
    return bigRuleList

def calcConf(freqSet , H , supportData , br1 , minConf = 0.7):
    prunedH = []
    
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet - conseq]
        
        if conf>=minConf:
            print freqSet - conseq , '>' , conseq , 'conf:' , conf
            br1.append((freqSet - conseq , conseq , conf))
            prunedH.append(conseq)
        
    return prunedH


def rulesFromConseq(freqSet , H , supportData , br1 , minConf = 0.7):
    m = len(H[0])
    
    if len(freqSet) > (m+1):
        Hmp1 = aprioriGen(H , m+1)
        Hmp1 = calcConf(freqSet , Hmp1 , supportData , br1 , minConf)
        
        if len(Hmp1) > 1:
            rulesFromConseq(freqSet , Hmp1 , supportData , br1 , minConf)

In [27]:
L , suppData = apriori(dataSet , minSupport=0.5)

rules = generateRules(L , suppData , minConf=0.7)

frozenset([1]) > frozenset([3]) conf: 1.0
frozenset([5]) > frozenset([2]) conf: 1.0
frozenset([2]) > frozenset([5]) conf: 1.0


In [28]:
rules

[(frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0)]

In [30]:
rules = generateRules(L , suppData , minConf=0.5)

frozenset([3]) > frozenset([1]) conf: 0.666666666667
frozenset([1]) > frozenset([3]) conf: 1.0
frozenset([5]) > frozenset([2]) conf: 1.0
frozenset([2]) > frozenset([5]) conf: 1.0
frozenset([3]) > frozenset([2]) conf: 0.666666666667
frozenset([2]) > frozenset([3]) conf: 0.666666666667
frozenset([5]) > frozenset([3]) conf: 0.666666666667
frozenset([3]) > frozenset([5]) conf: 0.666666666667
frozenset([5]) > frozenset([2, 3]) conf: 0.666666666667
frozenset([3]) > frozenset([2, 5]) conf: 0.666666666667
frozenset([2]) > frozenset([3, 5]) conf: 0.666666666667


In [31]:
rules

[(frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

In [32]:
#读入蘑菇数据
mushDatSet = [line.split() for line in open('../MLiA_SourceCode/Ch11/mushroom.dat').readlines()]

In [33]:
L , suppData = apriori(mushDatSet , minSupport=0.3)

In [34]:
for item in L[1]:
    if item.intersection('2'):
        print item

frozenset(['2', '59'])
frozenset(['39', '2'])
frozenset(['2', '67'])
frozenset(['2', '34'])
frozenset(['2', '23'])
frozenset(['2', '86'])
frozenset(['76', '2'])
frozenset(['90', '2'])
frozenset(['2', '53'])
frozenset(['93', '2'])
frozenset(['63', '2'])
frozenset(['2', '28'])
frozenset(['2', '85'])
frozenset(['2', '36'])


In [35]:
for item in L[3]:
    if item.intersection('2'):
        print item

frozenset(['39', '2', '53', '34'])
frozenset(['2', '59', '23', '85'])
frozenset(['2', '59', '90', '85'])
frozenset(['39', '2', '36', '34'])
frozenset(['39', '2', '90', '85'])
frozenset(['2', '59', '90', '86'])
frozenset(['2', '90', '93', '86'])
frozenset(['39', '2', '93', '34'])
frozenset(['76', '39', '2', '34'])
frozenset(['39', '2', '59', '86'])
frozenset(['2', '59', '90', '36'])
frozenset(['2', '59', '93', '85'])
frozenset(['2', '67', '86', '85'])
frozenset(['2', '28', '53', '85'])
frozenset(['2', '59', '93', '36'])
frozenset(['63', '23', '2', '34'])
frozenset(['2', '23', '36', '85'])
frozenset(['2', '90', '34', '85'])
frozenset(['39', '2', '59', '34'])
frozenset(['39', '2', '90', '34'])
frozenset(['2', '34', '28', '85'])
frozenset(['2', '59', '23', '36'])
frozenset(['2', '23', '93', '36'])
frozenset(['2', '23', '86', '85'])
frozenset(['39', '2', '34', '85'])
frozenset(['2', '93', '86', '34'])
frozenset(['63', '59', '2', '36'])
frozenset(['2', '67', '86', '34'])
frozenset(['39', '2'